# Приклад роботи з Pandas в середовищі для Jupyter notebook

#### 1. В середовищі Anaconda (меню:Environments) установіть пакет pandas-datareader

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import pandas_datareader as pdr
from statistics import *

####  2. Скачуємо інформацію про діяльність Google та Amazon за останні роки (Nasdaq-GS)

In [ ]:
amzn=pdr.get_data_yahoo('AMZN', '20121101')
goog=pdr.get_data_yahoo('GOOG', '2012-11-01')

#### 3. Перегляд зчитаних даних: початкова частина, кінцева частина, інформація

Зчитування за замовчуванням перших 5 рядків

In [ ]:
amzn.head()

Зчитування заданої кількості початкових рядків

In [ ]:
amzn.head(10)

Зчитування прикінцевих 5 рядків

In [ ]:
amzn.tail()

Інформація про набір даних, яка включає інформацію про кількість записів у стовпцях

In [ ]:
amzn.info()

Інформація про два набори даних, які зчитуємо одночасно

In [ ]:
print(amzn.info(), goog.info())

####  4. Фільтрування з використанням .loc,.iloc, index 

Відкриваємо тільки ціну акцій на момент закриття біржі

In [ ]:
amzn['Close']

Для більш точного виводу скористаємось методом .loc з параметрами. Перший параметр - це зріз, який вказує діапазон рядків, а другий параметр - це назва стовпця таблиці

In [ ]:
amzn.loc['20121110':'20121115','Open']

Фільтрація за допомогою методу .iloc. Параметрами цього методу є номер рядка і номер стовпця. Наприклад, виведемо перший рядок, третій стовпець 

In [ ]:
amzn.iloc[0,2]

Виводимо зріз за номерами рядків з першого по третій

In [ ]:
amzn.iloc[0:3,2]

Вивід зрізу по рядках і стовпцях одночасно

In [ ]:
amzn.iloc[0:3,2:4]

Фільтрація методом .index з умовою

In [ ]:
amzn[amzn.index>'20190801']

Вивід математичного очікування та дисперсії для стовпчика Volume з застосуванням форматування

In [ ]:
amzn[amzn.Volume>13910300] 

####  5. Фільтрація з попередніми обчисленнями

Обчислення математичного очікування та стандартного відхилення

In [ ]:
print('Mean: %.3f Std: %.3f' % (amzn['Volume'].mean(), amzn['Volume'].std()))

In [ ]:
high = amzn['Volume'].mean()+2*amzn['Volume'].std()

In [ ]:
len(amzn[amzn.Volume>high] )/len(amzn)

In [ ]:
print(amzn['Volume'].mean())

#### 6. Додавання стовпців до даних

Обрахуємо відсоток, на який змінюється ціна акцій на момент закриття біржі у порівнянні з попереднім днем. Результат записуємо у новий стовпчик таблиці

In [ ]:
amzn['percent'] = amzn.Close.pct_change()

In [ ]:
amzn.head()

####  7. Нарощування даних

Сформуємо набри даних для Google та Amazon за березень 2019 року

In [ ]:
goog19 = goog.loc['2019-03']
amzn19 = amzn.loc['2019-03']

In [ ]:
goog19.append(amzn19, sort=False).head()

Стовпчик 'percent' містить значення NaN. Відповідні значення не були обраховані для набору goog. Тому  дані  відсутні в наборі goog19. Цей факт можна визначити, скориставшись методом .info()

In [ ]:
goog19.append(amzn19, sort=False).info()

Ми бачимо, що у стовпчику 'percent' вдвоє менше даних, ніж в інших стовпчиках. Дані 'percent' відсутні у першій половині спільної таблиці. Але у другій половині, яка містить записи з набору amzn, такі дані наявні. Це можна перевірити, вивівши хвіст таблиці.

In [ ]:
goog19.append(amzn19, sort=False).tail()

#### 8. Порівняння стовпців даних з різних структур типу DataFrame

Порівняємо стовпці 'High' для наших наборів даних. Для цього створимо об'єкт high типу DataFrame. Як параметр передаємо словник з ключем, що відповідає набору даних, і значенням, що містить вектор стовпця. Результати роздрукували за допомогою методу .loc або іншого, відомого нам методу. 

In [ ]:
high=pd.DataFrame({'AMZN' : amzn.High, 'GOOG' : goog.High})
high.loc['20190110':'20190115','AMZN':'GOOG']

#### 9. Групування і сортування даних

Згрупувати можна записи, якщо вони мають однакові значення у стовпці, по якому проводимо групування. Отримаємо об'єкт зі згрупованими записами. До кожної такої групи можна застосувати функцію. Якщо у нашому наборі застосувати групування до стовпця 'Open', то будуть згруповані всі записи з однаковою ціною акцій на момент відкриття біржі. Застосуємо до згрупованих записів функцію sum().

In [ ]:
a=goog.groupby(['Open']).sum()
a.tail()

Очевидно, що ми не отримали групування, оскільки кожний день ціна акцій компанії на момент відкриття біржі була різна. Тому група у нас складається з одного запису. Стовпець, по якому проводилося групування, відображено жирним шрифтом.

Розглянемо узагальнення таблиці з групуванням. Нехай необхідно дізнатися середнє значення продажів у 2019 році помісячно. Для групування даних помісячно використаємо функцію resample() з параметром 'M'.

In [ ]:
goog['Volume'][goog.index>'2019'].resample('M').mean()

Інформація про середні продажі помісячно виведена в таблицю. Для більшої наглядності ми можемо представити її у вигляді графіка. Для цього додамо ще функцію plot().

In [ ]:
goog['Volume'][goog.index>'2019'].resample('M').mean().plot()

#### 10. Кумулятивна сума 

Розглянемо ще один вид агрегування даних. Нехай необхідно визначити графік поточної зміни мінімальної ціни на акції.
Для цього для кожного дня знайдемо відношення ціни акції до її ціни за попередній день.

In [ ]:
amzn_change=np.log(amzn.Low / amzn.Low.shift(1))
goog_change=np.log(goog.Low / goog.Low.shift(1))

Кумулятивну суму цих відношень покажемо на графіку

In [ ]:
amzn_change.cumsum().plot()
goog_change.cumsum().plot()

#### 11. Створення нового набору даних типу DataFrame і збереження його в csv

Створимо новий набір даних типу DataFrame. Цей набір міститиме дані amzn_change і goog.change. Оскільки порівняння відбувається з попереднім днем, то дані за перший день відсутні. Тому формування нашого набору починаємо з другого рядка.

In [ ]:
new=pd.DataFrame({'AMZN': amzn_change[1:], 'GOOG': goog_change[1:]})
new.head()

Записуємо набір даних new у файл 'stocks.csv'

In [ ]:
new.to_csv('stocks.csv')

Читаємо дані з файлу 'stocks.csv'

In [ ]:
new1=pd.read_csv('stocks.csv')
new1.head()

#### 12. Заміна індексного стовпця і видалення стовпців

При зчитуванні втрачається індексування по даті. Тому DataFrame додає новий стовпчик з номером, за яким відбувається індексування. Якщо необхідно позбутися цього, то існує два підходи.
Перший підхід, який перепризначає індекс:

In [ ]:
new1=pd.read_csv('stocks.csv')
new1.index=new1.Date
new1.head()

Другий підхід з використанням методу drop():

In [ ]:
new1.drop(['Date'],axis=1,inplace=True)
new1.head()

#### 13. Перейменування стовпців

In [ ]:
new1.rename(columns={'AMZN':'CH_AMZN'},inplace=True)
new1.head()

#### 14. Використання методу rolling() для обчислення пересувного середнього

In [ ]:
goog_close = pd.DataFrame(goog.Close).resample('W').mean()

In [ ]:
goog_close['MA_7'] = goog_close.Close.rolling(7).mean()
goog_close['MA_20'] = goog_close.Close.rolling(20).mean()

In [ ]:
plt.figure(figsize=(15,10))
plt.grid(True)
plt.plot(goog_close['Close'])
plt.plot(goog_close['MA_7'])
plt.plot(goog_close['MA_20'])